## Install dependencies

In [1]:
! pip uninstall -y transformers && pip install git+https://github.com/huggingface/transformers
! pip install ipywidgets

Found existing installation: transformers 4.38.0
Uninstalling transformers-4.38.0:
  Successfully uninstalled transformers-4.38.0
  Cloning https://github.com/huggingface/transformers to /private/var/folders/2m/ys1b248x6l93pxxf85kdct740000gn/T/pip-req-build-kwbjuyz4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/2m/ys1b248x6l93pxxf85kdct740000gn/T/pip-req-build-kwbjuyz4
  Resolved https://github.com/huggingface/transformers to commit b338a6c3b8eda29610d4d472cad8cd87cbfdaaed
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8660843 sha256=c68d8b6cdc5d537907c9990418718cf9f8f65b7d199eb76bb7c926b4ac501d60
  Stored in directory: /private/var/folders/2m/ys1b248x6l93pxxf85kdct740000gn/T/pip-ephem-wheel-cach

In [2]:
! pip install torch

  Using cached torch-2.2.1-cp311-none-macosx_11_0_arm64.whl.metadata (25 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.2.1-cp311-none-macosx_11_0_arm64.whl (59.7 MB)
Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


In [3]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer



### Selecting gpu based on system

In [4]:
def determine_device():
    """Determines the appropriate device to use for PyTorch based on the environment.

    Returns:
        str: The device name ('cuda', 'cpu', or 'mps').
    """
    # Check if running in Google Colab or Linux
    is_colab_or_linux = os.name == 'posix' and not os.path.exists('/System/Library')  # Checks for MacOS, not Linux

    # If running in Google Colab or Linux, check for CUDA availability
    if is_colab_or_linux:
        try:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        except ImportError:  # In case torch is not installed
            device = "cpu"
    else:  # Running on MacOS
        try:
            device = "mps" if torch.backends.mps.is_available() else "cpu"
        except ImportError:  # In case torch is not installed or MPS is not available
            device = "cpu"
    return device

device = determine_device()
print(device)


mps


In [5]:
# setting device
torch.set_default_device(device=device)

### Loading model 

In [6]:
model_id = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:


def generate_response(prompt: str, return_full_text: bool = True, split_key: str = "A:"):
    try:
        inputs = tokenizer(prompt, return_tensors='pt', return_attention_mask=False)
        outputs = model.generate(**inputs,
                                max_length=1000,
                                temperature=0.0,
                                top_p=0.95,
                                repetition_penalty=1.1,
                                top_k=10,
                                do_sample=False,
                                eos_token_id=tokenizer.eos_token_id,
                                pad_token_id=tokenizer.eos_token_id,
                                )
        full_response = tokenizer.batch_decode(outputs)[0]
        displayed_response = full_response if return_full_text else full_response.split(split_key)[-1].strip().replace("<|endoftext|>", "").strip()
        return displayed_response
    except Exception as e:
        return f"The following error occured: {e}."
    
    

# Question-answering

In [8]:
## Defining prompt templates
DEFAULT_SYSTEM_PROMPT = "Perfom the following instruction to the best of your ability."
QA_PROMPT_TEMPLATE = 'Q:{instruction}\nA:'
DEFAULT_PROMPT_TEMPLATE = "Q:{system_prompt}\n{instruction}\nA:"

PROMPT_TEMPLATE_SPACE = {
    "qa": [QA_PROMPT_TEMPLATE, "Instruct:{instruction}\nOutput:"],  # can add many more if needed
    "system_guided": [DEFAULT_PROMPT_TEMPLATE, "Instruct:{system_prompt}\n{instruction}\nOutput:"] # same here can add many more
}
# Choose prompt as PROMPT_TEMPLATE_SPACE.get("qa")[index] with index = 0  for example. So far the default prompt give better results.

In [9]:

# Generate answer from instruction without system prompt
def get_answer(instruction: str, include_prompt: bool = True) -> str:
    prompt_template = PROMPT_TEMPLATE_SPACE.get("qa")[0] # the first template gives better results
    prompt = prompt_template.format(instruction=instruction)
    return generate_response(prompt=prompt, return_full_text=include_prompt)


# Generate answer from instruction with system prompt
def get_system_guided_answer(instruction: str, include_prompt: bool = True) -> str:
    prompt_template = PROMPT_TEMPLATE_SPACE.get("system_guided")[0] # first template gives better results
    prompt = prompt_template.format(system_prompt=DEFAULT_SYSTEM_PROMPT, instruction=instruction) 
    return generate_response(prompt=prompt, return_full_text=include_prompt)





In [10]:
some_instructions = [
    "What is the capital city of Canada ?", # Ottawa
    "In which city can we find the Eiffel tower ?", #  Paris
    "Who was the US president between 2008 and 2012 ?", # Obama
    "Quel est l'actuel président de la république française ?", # Macron
    "What is the first prime number after 2024 ?", # 2027
    "Give me a python function that finds out if an integer is a prime number.",
    "Solve the following equation: 4x-10 = 6", #x =4,
    "Solve this equation: 4.5x-10 = -1.5" #x = 8.5/4.5 = 17/9 = 1.888888...

]


In [11]:

for instruction in some_instructions:
    print(get_answer(instruction=instruction))
    print()
    print(get_system_guided_answer(instruction=instruction))
    print(3*"\n")
    



/Users/hb/existing_repos/llms-notebooks/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/hb/existing_repos/llms-notebooks/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/Users/hb/existing_repos/llms-notebooks/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.w

Q:What is the capital city of Canada?
A: Ottawa.
<|endoftext|>

Q:Perfom the following instruction to the best of your ability.
What is the capital city of Canada?
A:The capital city of Canada is Ottawa.
<|endoftext|>




Q:In which city can we find the Eiffel tower?
A: Paris.
<|endoftext|>

Q:Perfom the following instruction to the best of your ability.
In which city can we find the Eiffel tower?
A: Paris, France is where the Eiffel Tower is located.
<|endoftext|>




Q:Who was the US president between 2008 and 2012?
A:Barack Obama.
<|endoftext|>

Q:Perfom the following instruction to the best of your ability.
Who was the US president between 2008 and 2012?
A:Barack Obama was the US President between 2008 and 2012.
<|endoftext|>




Q:Quel est l'actuel président de la république française?
A:L'actuel président de la république française est Emmanuel Macron.
<|endoftext|>

Q:Perfom the following instruction to the best of your ability.
Quel est l'actuel président de la république franç

- The prompt template `'Q:{instruction}\nA:'` gives a very short answer compare to the system-guided prompt template `"Q:{system_prompt}\n{instruction}\nA:"` where `system_prompt="Perfom the following instruction to the best of your ability."`

- Note that for solving equations it doesn't perform well when we use decimal numbers. This is not surprising at all. 

## Summarization

In [12]:
def summarize(input_text: str) -> str:
    prompt = f"##INPUT: Give me a summary of the following: {input_text}\n##OUTPUT:"
    return generate_response(prompt=prompt)

input_text = """Cher membre Amazon Prime,

Nous vous écrivons pour vous informer des nouveautés concernant votre expérience Prime Video. À partir du 9 avril 2024, les films et séries Prime Video incluront de la publicité en quantité limitée*. Cela nous permettra de continuer à investir dans des contenus attractifs et d'augmenter nos investissements sur le long terme, afin de maintenir la qualité et la quantité des contenus sur Prime Video. Nous visons à avoir sensiblement moins de publicités que la télévision linéaire classique et les autres services de streaming. Il n'y a aucun changement concernant le prix actuel de votre abonnement Prime. Vous pouvez annuler votre abonnement gratuitement ou consulter votre prochaine date de renouvellement en visitant votre compte ici. Nous proposerons également une nouvelle option sans publicité pour €1,99 de plus par mois** à laquelle vous pouvez vous abonner ici.
"""

print(summarize(input_text=input_text))

##INPUT: Give me a summary of the following: Cher membre Amazon Prime,

Nous vous écrivons pour vous informer des nouveautés concernant votre expérience Prime Video. À partir du 9 avril 2024, les films et séries Prime Video incluront de la publicité en quantité limitée*. Cela nous permettra de continuer à investir dans des contenus attractifs et d'augmenter nos investissements sur le long terme, afin de maintenir la qualité et la quantité des contenus sur Prime Video. Nous visons à avoir sensiblement moins de publicités que la télévision linéaire classique et les autres services de streaming. Il n'y a aucun changement concernant le prix actuel de votre abonnement Prime. Vous pouvez annuler votre abonnement gratuitement ou consulter votre prochaine date de renouvellement en visitant votre compte ici. Nous proposerons également une nouvelle option sans publicité pour €1,99 de plus par mois** à laquelle vous pouvez vous abonner ici.

##OUTPUT: Amazon Prime is informing you about new featu